# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

2025-10-07 19:03:14.172330: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [5]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [6]:
# Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))
# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))


# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])


# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 10/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - 

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [7]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [8]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [9]:
# Train the model
normalized_input_model.fit(X_train,  
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))# Train the model

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 10/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step -

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [10]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [11]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))


# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train,  
                           y_train_scaled, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.5368 - mse: 0.5368 - val_loss: 0.1869 - val_mse: 0.1869
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2506 - mse: 0.2506 - val_loss: 0.1454 - val_mse: 0.1454
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.2079 - mse: 0.2079 - val_loss: 0.1299 - val_mse: 0.1299
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1879 - mse: 0.1879 - val_loss: 0.1251 - val_mse: 0.1251
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1712 - mse: 0.1712 - val_loss: 0.1225 - val_mse: 0.1225
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1514 - mse: 0.1514 - val_loss: 0.1142 - val_mse: 0.1142
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1410 - mse: 0.1410 - val_loss: 0.1131 - val_mse: 0.1131
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1336 - mse: 0.1336 - val_loss: 0.1082 - val_mse: 0.1082
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.12

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [12]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0104 - mse: 0.0104


[0.010415523312985897, 0.010415523312985897]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [13]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1067 - mse: 0.1067 


[0.1067083403468132, 0.1067083403468132]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [14]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


25670.282273477245

Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [15]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features)))
# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))
# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))


# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])


# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.4828 - mse: 0.4828 - val_loss: 0.2410 - val_mse: 0.2410
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.2393 - mse: 0.2393 - val_loss: 0.1919 - val_mse: 0.1919
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1998 - mse: 0.1998 - val_loss: 0.1522 - val_mse: 0.1522
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1699 - mse: 0.1699 - val_loss: 0.1397 - val_mse: 0.1397
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1526 - mse: 0.1526 - val_loss: 0.1352 - val_mse: 0.1352
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1376 - mse: 0.1376 - val_loss: 0.1353 - val_mse: 0.1353
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1293 - mse: 0.1293 - val_loss: 0.1282 - val_mse: 0.1282
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1189 - mse: 0.1189 - val_loss: 0.1324 - val_mse: 0.1324
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.11

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [16]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0093 - mse: 0.0093


[0.009260429069399834, 0.009260429069399834]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [17]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1478 - mse: 0.1478 


[0.1478227823972702, 0.1478227823972702]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [18]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=n_features))
# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))
# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))


# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.3950 - mse: 0.3950 - val_loss: 0.1616 - val_mse: 0.1616
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2268 - mse: 0.2268 - val_loss: 0.1364 - val_mse: 0.1364
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1856 - mse: 0.1856 - val_loss: 0.1226 - val_mse: 0.1226
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1588 - mse: 0.1588 - val_loss: 0.1157 - val_mse: 0.1157
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1387 - mse: 0.1387 - val_loss: 0.1123 - val_mse: 0.1123
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1289 - mse: 0.1289 - val_loss: 0.1069 - val_mse: 0.1069
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1134 - mse: 0.1134 - val_loss: 0.1055 - val_mse: 0.1055
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1083 - mse: 0.1083 - val_loss: 0.1029 - val_mse: 0.1029
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.10

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0062 - mse: 0.0062


[0.006246396340429783, 0.006246396340429783]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [20]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1065 - mse: 0.1065 


[0.1065259724855423, 0.1065259724855423]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [21]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', 
                      loss='mse', 
                      metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.3261 - mse: 0.3261 - val_loss: 0.1206 - val_mse: 0.1206
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1576 - mse: 0.1576 - val_loss: 0.1119 - val_mse: 0.1119
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1157 - mse: 0.1157 - val_loss: 0.1540 - val_mse: 0.1540
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0936 - mse: 0.0936 - val_loss: 0.1322 - val_mse: 0.1322
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0751 - mse: 0.0751 - val_loss: 0.0938 - val_mse: 0.0938
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0582 - mse: 0.0582 - val_loss: 0.1006 - val_mse: 0.1006
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0534 - mse: 0.0534 - val_loss: 0.1060 - val_mse: 0.1060
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0424 - mse: 0.0424 - val_loss: 0.1471 - val_mse: 0.1471
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.04

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0040 - mse: 0.0040


[0.004021300934255123, 0.004021300934255123]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [25]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0999 - mse: 0.0999 


[0.0999029278755188, 0.0999029278755188]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [26]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam', 
                   loss='mse', 
                   metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.3841 - mse: 0.3841 - val_loss: 0.1297 - val_mse: 0.1297
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1529 - mse: 0.1529 - val_loss: 0.1019 - val_mse: 0.1019
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0996 - mse: 0.0996 - val_loss: 0.1037 - val_mse: 0.1037
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0750 - mse: 0.0750 - val_loss: 0.1044 - val_mse: 0.1044
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0576 - mse: 0.0576 - val_loss: 0.1131 - val_mse: 0.1131
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0490 - mse: 0.0490 - val_loss: 0.1071 - val_mse: 0.1071
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0350 - mse: 0.0350 - val_loss: 0.1266 - val_mse: 0.1266
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0301 - mse: 0.0301 - val_loss: 0.1061 - val_mse: 0.1061
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [27]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0039 - mse: 0.0039


[0.0038529513403773308, 0.0038529513403773308]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [28]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1063 - mse: 0.1063 


[0.10633614659309387, 0.10633614659309387]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [30]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.1830 - mse: 0.1830


[0.18304650485515594, 0.18304650485515594]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [31]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


33621.12288428214

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.